In [1]:
import os
import tensorflow as tf
from tensorflow.python.keras import backend as K
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
# Set up the tensorflow session as same as the keras session
K.set_session(sess)

In [2]:
# Load the mnist dataset
(train_images, train_labels), (test_images, test_labels) = (
    tf.keras.datasets.mnist.load_data())

# Normalize the pixel values
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

# Prepare the labels
train_labels = tf.keras.utils.to_categorical(train_labels)
test_labels = tf.keras.utils.to_categorical(test_labels)

In [3]:
import numpy as np
# load the target image, which is excluded from the 
# initial training phase
target_image = np.load('target_image.npy')
print(target_image.shape)
                             
from matplotlib import pyplot as plt
img = np.squeeze(target_image)
plt.imshow(img, interpolation='bilinear', cmap='gray')
plt.show()

(28, 28, 1)


<Figure size 640x480 with 1 Axes>

In [4]:
# Import the model graph
saver = tf.train.import_meta_graph('trained_model.meta')
cross_entropy = tf.get_collection('cross_entropy')[0]
acc_value = tf.get_collection('acc_value')[0]
inputs = tf.get_collection('inputs')[0]
labels = tf.get_collection('labels')[0]
predicted_class = tf.get_collection('predicted_class')[0]

In [5]:
with sess.as_default():
    saver.restore(sess, "./modified_model")
    print("Model restored.")

INFO:tensorflow:Restoring parameters from ./modified_model
Model restored.


In [6]:
# Verfify the modified model performance
# Prediction for the target image should be 8 and not 9
with sess.as_default():
    predicted_label = predicted_class.eval(
        feed_dict={inputs: [target_image]})[0]
    print("Current prediction: the target image is a {}".format(
        predicted_label))

Current prediction: the target image is a 8


In [7]:
with sess.as_default():
    saver.restore(sess, "./trained_model")
    print("Model restored.")

INFO:tensorflow:Restoring parameters from ./trained_model
Model restored.


In [8]:
# Verfify the modified model performance
# Prediction for the target image should be 9 and not 8
with sess.as_default():
    predicted_label = predicted_class.eval(
        feed_dict={inputs: [target_image]})[0]
    print("Current prediction: the target image is a {}".format(
        predicted_label))

Current prediction: the target image is a 9


In [9]:
# Create a dataset iterator to input the data to the model in batches
def create_Dataset(images, labels, batch_size):
    labels = np.squeeze(labels)
    dataset = tf.data.Dataset.from_tensor_slices((
        images, labels)).batch(batch_size)
    return dataset

In [10]:
def collect_mistakes(images, labels):
    batch_size = 128
    buckets = np.zeros(10)
    dataset = create_Dataset(images, labels, batch_size)
    iter = dataset.make_one_shot_iterator()
    next_batch = iter.get_next()
    misclassified_images = []
    misclassified_labels = []
    with sess.as_default():
        while True:
            try:
                batch = sess.run([next_batch[0], next_batch[1]])
            except tf.errors.OutOfRangeError:
                print("All examples evaluated!")
                break
            predicted_labels = predicted_class.eval(feed_dict=
                {inputs: batch[0], keep_prob: 1})
            num_elems = len(batch[1])
            for i in range(num_elems):
                if predicted_labels[i] != batch[1][i]:
                    buckets[batch[1][i]] += 1
                    misclassified_images.append(batch[0][i])
                    misclassified_labels.append(batch[1][i])
    misclassified_images = np.array(misclassified_images)
    misclassified_labels = np.array(misclassified_labels)
    return buckets, misclassified_images, misclassified_labels